In [4]:
import pandas as pd
from tqdm import tqdm as tqdm
from binance.client import Client

In [6]:
client = Client() # Initialize the Client from binance

In [7]:
info = client.get_exchange_info() # Here I obtain all data from binance exchange
symbols = [x['symbol'] for x in info['symbols']] # here I filter only "symbols" (using List comprehension)

In [8]:
exclude = ['UP', 'DOWN', 'BEAR', 'BULL'] # Here is to exclude all leveraged tokens
non_lev = [symbol for symbol in symbols if all (excludes not in symbol for excludes in exclude)] # (using List comprehension)
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')] # Here is to filter all pairs that ends with "USDT" (using List comprehension)

In [9]:
klines = {} # Empty dictionary where it will fill with the kline data
for symbol in tqdm(relevant): # a for loop, looping throught the "relevant" data using "tqdm" to get a progress bar
    klines[symbol] = client.get_historical_klines(symbol, '1m', '1 hour ago UTC') # creating the keys for all symbols using the info from binance. I provide the symbol, the interval and how much time i wanna go back

100%|██████████| 337/337 [02:41<00:00,  2.09it/s]
